# Terra em Foco Conference: Introduction to Sentinel Hub APIs in CDSE: Part Two

*This notebook was written for a two hour training session at the Terra em Foco, the Portuguese Earth Observation National Conference held in Braga, Portugal 12-13th September 2024.*

The Copernicus Data Space Ecosystem offers immediate access to large amounts of open and free Earth observation data from the Copernicus Sentinel satellites, including both new and historical Sentinel images, as well as Copernicus Contributing Missions. As the effects of climate change intensifies, the use of earth observation data will become ever more important to monitor wildfires in the southern Europe including Portugal. These two notebooks will show how you can utilise Copernicus data to do just this.

The Copernicus Data Space Ecosystem supports users in accessing, viewing, using, downloading, and analyzing data. The Copernicus Data Space Ecosystem is set up to further improve access and exploitation of the EU’s Copernicus satellites data. The service aims to support users in building various applications needed to provide accurate, timely and objective information which are crucial to create a more sustainable future.

The Copernicus Data Space Ecosystem offers multiple Application Programming Interfaces (APIs) ranging from catalogue, product download, visualization over processing web services such as STAC, openEO and Sentinel Hub APIs. This Jupyter notebook focuses on the Sentinel Hub APIs The Sentinel Hub API is a RESTful API interface that provides access to various satellite imagery archives. It allows you to access raw satellite data, rendered images, statistical analysis, and other features.

You will be shown how to:

- Monitor the recovery of vegetation post event visually
- Delineate and then polygonise the burn scar using Process API and Python
- Use the polygon to create to create time series of vegetation indices.

**Note:** You should already have some experience of Sentinel Hub APIs to be able to follow the functions in this notebook. You can begin by going through part one of the workshop. However, it is advised that you should have some basic knowledge of python and its data science libraries.

Firstly, before getting started we should import some libraries. In addition, to common data science libraries we will be utilising the [Sentinel Hub Python package](https://sentinelhub-py.readthedocs.io/en/latest/index.html). This comes preinstalled in the Copernicus Data Space Ecosystem Jupyter Lab. 

In [ ]:
# General utilities
import getpass
import warnings
from pathlib import Path
from typing import Any, Optional, Tuple
import datetime

# Plotting
import geopandas as gpd
import matplotlib.patches as mpatches
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

# Reading satellite imagery
import rasterio as rio
from matplotlib import colors
from rasterio.mask import mask
from rasterio.merge import merge
from rasterio.warp import calculate_default_transform, reproject
from rasterio.features import shapes
from rasterio.features import sieve

# Sentinel Hub services
from sentinelhub import (
    CRS,
    DataCollection,
    Geometry,
    MimeType,
    SentinelHubRequest,
    SentinelHubStatistical,
    SHConfig,
    SentinelHubDownloadClient,
)
from shapely.geometry import shape

warnings.filterwarnings("ignore")
%matplotlib inline

In the following cell, we have defined some python functions to help us process and visualise the data later on in the notebook:

In [ ]:
def plot_image(
    image: np.ndarray,
    factor: float = 1.0,
    clip_range: Optional[Tuple[float, float]] = None,
    **kwargs: Any,
) -> None:
    """Utility function for plotting RGB images."""
    fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(15, 15))
    if clip_range is not None:
        ax.imshow(np.clip(image * factor, *clip_range), **kwargs)
    else:
        ax.imshow(image * factor, **kwargs)
    ax.set_xticks([])
    ax.set_yticks([])


def vectorize_raster(raster_path, vector_path):
    with rio.open(raster_path) as src:
        raster_data = src.read(1)  # read only the first band
        transform = (
            src.transform
        )  # to convert pixel coordinates to geographic coordinates
        crs = src.crs

    shapes_generator = shapes(
        raster_data, transform=transform
    )  # add mask=raster_data != 0 to exclude nodata values

    vector_data = [
        {"properties": {"value": value}, "geometry": shape}
        for shape, value in shapes_generator
    ]

    gdf = gpd.GeoDataFrame.from_features(vector_data, crs=crs)
    # save to file
    # gdf.to_file(vector_path, driver="GeoJSON")
    return gdf


def reproject_raster(inpath, outpath, crs, method="nearest"):
    "Reproject a raster to a new coordinate system."

    dst_crs = f"EPSG:{crs}"

    with rio.open(inpath) as src:
        transform, width, height = calculate_default_transform(
            src.crs, dst_crs, src.width, src.height, *src.bounds
        )
        kwargs = src.meta.copy()
        kwargs.update(
            {"crs": dst_crs, "transform": transform, "width": width, "height": height}
        )

        with rio.open(outpath, "w", **kwargs) as dst:
            for i in range(1, src.count + 1):
                reproject(
                    source=rio.band(src, i),
                    destination=rio.band(dst, i),
                    src_transform=src.transform,
                    src_crs=src.crs,
                    dst_transform=transform,
                    dst_crs=dst_crs,
                    resampling=rio_resample(method),
                )


def write_raster(path, raster, crs, transform, nodata, driver="GTiff"):
    """Write a raster to a file."""

    with rio.open(
        path,
        "w",
        driver=driver,
        height=raster.shape[0],
        width=raster.shape[1],
        count=1,
        dtype=raster.dtype,
        crs=crs,
        transform=transform,
        nodata=nodata,
    ) as dst:
        dst.write(raster, 1)


# define functions to extract statistics for all acquisition dates


def extract_stats(date, stat_data):
    d = {}
    for key, value in stat_data["outputs"].items():
        stats = value["bands"]["B0"]["stats"]
        if stats["sampleCount"] == stats["noDataCount"]:
            continue
        else:
            d["date"] = [date]
            for stat_name, stat_value in stats.items():
                if stat_name == "sampleCount" or stat_name == "noDataCount":
                    continue
                else:
                    d[f"{key}_{stat_name}"] = [stat_value]
    return pd.DataFrame(d)


def read_acquisitions_stats(stat_data):
    df_li = []
    for aq in stat_data:
        date = aq["interval"]["from"][:10]
        df_li.append(extract_stats(date, aq))
    return pd.concat(df_li)

## Authentication

You can obtain the credentials for the Sentinel Hub services (`client_id` & `client_secret`) in your [Dashboard](https://shapps.dataspace.copernicus.eu/dashboard/#/). In the user settings, you can create a new OAuth client to generate these credentials. You can find more detailed instructions on the corresponding [documentation page](https://documentation.dataspace.copernicus.eu/APIs/SentinelHub/Overview/Authentication.html).

Now that you have your `client_id` & `client_secret`, it is recommended to configure a new profile in your Sentinel Hub Python package. Instructions for configuring your Sentinel Hub Python package can be found [here](https://sentinelhub-py.readthedocs.io/en/latest/configure.html). Using these instructions, you can create a profile specifically tailored to use the package to access the Copernicus Data Space Ecosystem data collections. This is useful because changes to the Config class in your notebook are usually only temporary. If you save the configuration in your profile, you do not have to generate new credentials or overwrite/change the default profile every time you start or write a new Jupyter notebook.

If you are using the Sentinel Hub Python package for the Copernicus Data Space Ecosystem for the first time, you should create a profile specifically for the Copernicus Data Space Ecosystem. You can do this in the following cell:

In [ ]:
# Only run this cell if you have not created a configuration.

config = SHConfig()
# config.sh_client_id = getpass.getpass("Enter your SentinelHub client id")
# config.sh_client_secret = getpass.getpass("Enter your SentinelHub client secret")
config.sh_token_url = "https://identity.dataspace.copernicus.eu/auth/realms/CDSE/protocol/openid-connect/token"
config.sh_base_url = "https://sh.dataspace.copernicus.eu"
# config.save("cdse")

However, if you have already configured a profile in Sentinel Hub Python for the Copernicus Data Space Ecosystem, then you can run the below cell entering the profile name as a string replacing `<profile_name>`.

In [ ]:
# config = SHConfig("cdse")

We are in a new notebook so we need to redefine our AOI here:

In [ ]:
area_of_interest = """{
  "type": "Polygon",
  "coordinates": [
    [
      [
        -8.920627,
        37.362517
      ],
      [
        -8.516631,
        37.362517
      ],
      [
        -8.516631,
        37.628372
      ],
      [
        -8.920627,
        37.628372
      ],
      [
        -8.920627,
        37.362517
      ]
    ]
  ]
}
"""

aoi = gpd.read_file(area_of_interest)
aoi["geometry"] = aoi
aoi["area"] = aoi.area
aoi.explore("area", color="Green", legend=False)

## 4: Monitoring the recovery of vegetation post event

In [ ]:
start = datetime.datetime(2023, 8, 10)
end = datetime.datetime(2024, 7, 10)
n_chunks = 10
tdelta = (end - start) / n_chunks
edges = [(start + i * tdelta).date().isoformat() for i in range(n_chunks)]
slots = [(edges[i], edges[i + 1]) for i in range(len(edges) - 1)]

print("Time windows:\n")
for slot in slots:
    print(slot)

In [ ]:
full_geometry = Geometry(aoi.to_crs(32630).geometry.values[0], crs=CRS.UTM_30N)

evalscript_swir_composite = """
//VERSION=3
function setup() {
  return {
    input: ["B12","B8A","B04", "dataMask"],
    output: { bands: 4 }
  };
}

function evaluatePixel(sample) {
  return [sample.B12,sample.B8A,sample.B04, sample.dataMask ];
}
"""


def get_swir_composite_request(time_interval):
    return SentinelHubRequest(
        evalscript=evalscript_swir_composite,
        input_data=[
            SentinelHubRequest.input_data(
                data_collection=DataCollection.SENTINEL2_L2A.define_from(
                    name="s2", service_url="https://sh.dataspace.copernicus.eu"
                ),
                time_interval=time_interval,
                other_args={"dataFilter": {"mosaickingOrder": "leastCC"}},
            )
        ],
        responses=[SentinelHubRequest.output_response("default", MimeType.PNG)],
        geometry=full_geometry,
        size=[1000, 920],
        config=config,
    )

In [ ]:
# create a list of requests
list_of_requests = [get_swir_composite_request(slot) for slot in slots]
list_of_requests = [request.download_list[0] for request in list_of_requests]

# download data with multiple threads
data = SentinelHubDownloadClient(config=config).download(
    list_of_requests, max_threads=5
)

In [ ]:
# some stuff for pretty plots
ncols = 3
nrows = 3
aspect_ratio = 1000 / 920
subplot_kw = {"xticks": [], "yticks": [], "frame_on": False}

fig, axs = plt.subplots(
    ncols=ncols,
    nrows=nrows,
    figsize=(5 * ncols * aspect_ratio, 5 * nrows),
    subplot_kw=subplot_kw,
)

for idx, image in enumerate(data):
    ax = axs[idx // ncols][idx % ncols]
    ax.imshow(np.clip(image * 2.5 / 255, 0, 1))
    ax.set_title(f"{slots[idx][0]}  -  {slots[idx][1]}", fontsize=10)

plt.tight_layout()

## 5. Delineate and then polygonize the burn scar

In [ ]:
# Set the directory where results will be stored
results_dir = "./data"

In [ ]:
evalscript_burn_scar_detection = """
//VERSION=3
// Burneed area detection
// Author: Monja B. Šebela

function setup() {
    return {
        input: ["B02", "B03", "B04", "B08", "B11", "B12", "dataMask"],
        output: { bands: 3, sampleType:"UINT8" }
      };
}

function evaluatePixel(samples) {
	var NDWI=index(samples.B03, samples.B08); 
	var NDVI=index(samples.B08, samples.B04);
	var INDEX= ((samples.B11 - samples.B12) / (samples.B11 + samples.B12))+(samples.B08);

  	if((INDEX>0.2)||(samples.B02>0.1)||(samples.B11<0.1)||(NDVI>0.3)||(NDWI > 0.1)){
  		return[0,0,0]
	}
	else {
 	return[1,1,1]
	}
}
"""

request = SentinelHubRequest(
    evalscript=evalscript_burn_scar_detection,
    input_data=[
        SentinelHubRequest.input_data(
            data_collection=DataCollection.SENTINEL2_L2A.define_from(
                name="s2", service_url="https://sh.dataspace.copernicus.eu"
            ),
            time_interval=("2023-08-12", "2023-08-12"),
            other_args={"dataFilter": {"mosaickingOrder": "leastRecent"}},
        )
    ],
    responses=[SentinelHubRequest.output_response("default", MimeType.TIFF)],
    geometry=full_geometry,
    size=[1000, 920],
    data_folder=results_dir,
    config=config,
)

The method `get_data()` will always return a list of length 1 with the available image from the requested time interval in the form of numpy arrays.

In [ ]:
sh_request = request.get_data(save_data=True)

#### Clean up the burn scar

One of the limitations of Sentinel Hub services is that it fumctions on a per-pixel basis. Therefore to perform a morphological operation on the results we will open the raster and apply the last step on the downloaded file.

We then open the raster with `rasterio` and clean the data as we did in the "traditional" approach. We save the output, as well as the reprojected version to be able to plot the data interactively.

In [ ]:
burn_scar_raster = request.get_filename_list()

Next, we can sieve the raster to remove noise from the result:

In [ ]:
burn_scar = rio.open(f"data/%s" % burn_scar_raster[0])

burn_scar_cleaned = sieve(burn_scar, size=400)

# Save the output
write_raster(
    f"{results_dir}/burn_area.tif",
    burn_scar_cleaned.astype("uint8"),
    burn_scar.crs,
    burn_scar.transform,
    0,
)

# Reproject to EPSG 4326
reproject_raster(
    f"{results_dir}/burn_area.tif",
    f"{results_dir}/burn_area_4326.tif",
    3857,
    method="nearest",
)

burn_scar.close()

In [ ]:
cmap_burn_scar = colors.ListedColormap(["green", "red"])
bounds_burn_scar = [0, 0.5, 1]
norm_burn_scar = colors.BoundaryNorm(bounds_burn_scar, cmap_burn_scar.N)

fig, ax = plt.subplots(figsize=(12, 8))
im = ax.imshow(burn_scar_cleaned, cmap=cmap_burn_scar, norm=norm_burn_scar)
ax.set_xticks([])
ax.set_yticks([])
ax.legend(
    handles=[
        mpatches.Patch(color="green", label="Not burned"),
        mpatches.Patch(color="red", label="Burned"),
    ]
)
ax.set_title("Cleaned Burn Scar")

plt.show()

Once happy with the result, we can vectorise the raster so that we can use it as an input in the next step in the workflow.

In [ ]:
raster_path = "data/burn_area_4326.tif"
vector_path = "data/vectorized.geojson"

burn_scar = vectorize_raster(raster_path, vector_path)

If we examine the burn scar vector, we can currently see that it is made up of several features. To simplify this we can dissolve it into a single Multipolygon feature.

In [ ]:
burn_scar

In [ ]:
burn_scar["area"] = burn_scar.area
burn_scar["area"]
burn_scar = burn_scar[burn_scar["value"] == 1.0]
burn_scar = burn_scar[burn_scar["area"] >= 0]
burn_scar = burn_scar.dissolve()
burn_scar

In [ ]:
burn_scar.explore("value", color="Green", legend=False)

Now we're ready to use the polygon as an input into a Statistical API request.

## 6. Plot an mean NDVI and mean Burn Ratio timeseries using the burnscar post event

We will do this using Statistical API. In the following request, we actually request two different indices in two seperate outputs.

In [ ]:
full_geometry = Geometry(burn_scar.to_crs(32630).geometry.values[0], crs=CRS.UTM_30N)

In [ ]:
yearly_time_interval = "2023-07-01", "2024-07-01"

ndvi_evalscript = """
//VERSION=3

function setup() {
  return {
    input: [
      {
        bands: [
          "B04",
          "B06",
          "B07",
          "B08",
          "B8A",
          "B12",
          "dataMask"
        ]
      }
    ],
    output: [
      {
        id: "ndvi",
        bands: 1
      },
      {
        id: "burnratio",
        bands: 1
      },  
      {
        id: "dataMask",
        bands: 1
      }
    ]
  }
}


function evaluatePixel(samples) {

    return {
      ndvi: [index(samples.B08, samples.B04)],
      burnratio: [index(samples.B08, samples.B12)],
      dataMask: [samples.dataMask]
    };
    }
"""

aggregation = SentinelHubStatistical.aggregation(
    evalscript=ndvi_evalscript,
    time_interval=yearly_time_interval,
    aggregation_interval="P5D",
    resolution=(10, 10),
)

input_data = SentinelHubStatistical.input_data(
    DataCollection.SENTINEL2_L2A.define_from("s2", service_url=config.sh_base_url),
    other_args={"dataFilter": {"maxCloudCoverage": 10}},
)

ndvi_requests = []

request = SentinelHubStatistical(
    aggregation=aggregation,
    input_data=[input_data],
    geometry=full_geometry,
    config=config,
)
response1 = request.get_data()
response1

In [ ]:
result_df1 = read_acquisitions_stats(response1[0]["data"])
result_df1

We can take this another step further, and display the data in a time series using the Matplotlib python library:

In [ ]:
fig_stat, ax_stat = plt.subplots(1, 1, figsize=(18, 6))
t1 = result_df1["date"]
t2 = result_df1["date"]
ndvi_mean_field1 = result_df1["ndvi_mean"]
ndvi_std_field1 = result_df1["ndvi_stDev"]
ndvi_mean_field2 = result_df1["burnratio_max"]
ndvi_std_field2 = result_df1["burnratio_stDev"]
ax_stat.plot(t1, ndvi_mean_field1, label="NDVI mean")
ax_stat.fill_between(
    t1,
    ndvi_mean_field1 - ndvi_std_field1,
    ndvi_mean_field1 + ndvi_std_field1,
    alpha=0.3,
    label="NDVI stDev",
)
ax_stat.plot(t2, ndvi_mean_field2, label="Burn Ratio mean")
ax_stat.fill_between(
    t2,
    ndvi_mean_field2 - ndvi_std_field2,
    ndvi_mean_field2 + ndvi_std_field2,
    alpha=0.3,
    label="Burn Ratio stDev",
)
ax_stat.tick_params(axis="x", labelrotation=30, labelsize=12)
ax_stat.tick_params(axis="y", labelsize=12)
ax_stat.set_xlabel("Date", size=15)
ax_stat.set_ylabel("NDVI/unitless", size=15)
ax_stat.legend(loc="lower right", prop={"size": 12})
ax_stat.set_title("NDVI time series", fontsize=20)
for label in ax_stat.get_xticklabels()[1::20]:
    label.set_visible(False)

## 7. Exercise: Compare burned area with an area of unburned vegetation

Now that you have learned how you visualise time series derived from statistical API, you can apply your knowledge to try some different comparisons; here are some hints on what you could try out:
- create another AOI - you can visit the Request Builder application, draw an AOI and then download it there. 
- put together a new request with the new AOI
- change the date range and/or the interval period. try changing the cloud cover percentage used.
- update the previous cell to plot the two AOIs onto the same plot.